# Light interception

In [ ]:
%matplotlib notebook
from openalea.rose.experiment import *
from oawidgets.plantgl import PlantGL
from openalea.plantgl.all import *
from openalea.rose.data import manips, experiments
from ipywidgets import interactive, Dropdown, Checkbox, Tab, interact_manual

from openalea.spice.simulator import *
from openalea.spice import Scene as spScene
from openalea.spice.common.convert import pgl_to_spice, spice_add_pgl

### Experiment 2

In [5]:
def select_disposition(configuration):
    pgl_scene = sensor_exp(1) 
    env = environment()+ experiment2(configuration)

    simulator = Simulator()
    simulator.configuration.NB_PHOTONS = int(1e7)
    simulator.configuration.SCALE_FACTOR = 10
    simulator.configuration.MAXIMUM_DEPTH = 50
    simulator.configuration.T_MIN = 0.01
    simulator.configuration.BACKFACE_CULLING = False
    simulator.configuration.KEEP_ALL = False

    # setup environmentphmap
    sp_scene = pgl_to_spice(pgl_scene, sensors=True, setup=False)
    sp_scene = spice_add_pgl(sp_scene, env, sensors=False, setup=True)
    shapes = []
    for sh in pgl_scene:
        if isinstance(sh.geometry, pgl.Text):
            continue
        shapes.append(sh)
    simulator.addFaceSensors(shapes)
    
    simulator.scene_pgl = pgl_scene + env
    simulator.scene = sp_scene

    # S2 series of lamps
    simulator.addPointLight(Vec3(730, 1670, 2100), 400)
    simulator.addPointLight(Vec3(730, 176, 2100), 400)
    simulator.addPointLight(Vec3(1670, 176, 2100), 400)
    simulator.addPointLight(Vec3(1670, 1660, 2100), 400)
    
    # run
    res = simulator.run()
    sc = simulator.visualizePhotons(mode="oawidgets")
    rays = simulator.visualizeRays(color=0x33d17a, opacity=0.105)
    cmap = simulator.visualizeResults(mode='oawidgets')

    tab = Tab()
    tab.children = [sc, rays, cmap]
    tab.titles = ["Photons", "Rays", "Colormap"]
    display(tab)
    
    return simulator

disposition=Dropdown(options=["0","1","2","3","4"], value="1")

In [ ]:
#w = interactive(select_disposition, {"manual": True}, configuration=disposition)
#w

In [ ]:
# sim = w.result
# sim.results.display_face_sensor()

### Experiment 3

In [ ]:
def select_experiment(manip_name="High_Light_Crop_1", stage="FBV", fill=True):
    pgl_scene = manip_named(manip_name, stage, fill)
    env = environment()

    simulator = Simulator()
    simulator.configuration.NB_PHOTONS = int(1e6)
    simulator.configuration.SCALE_FACTOR = 10
    simulator.configuration.MAXIMUM_DEPTH = 50
    simulator.configuration.T_MIN = 0.01
    simulator.configuration.BACKFACE_CULLING = False
    simulator.configuration.KEEP_ALL = False

    # setup environmentphmap
    sp_scene = pgl_to_spice(pgl_scene, sensors=True, setup=False)
    sp_scene = spice_add_pgl(sp_scene, env, sensors=False, setup=True)
    shapes = []
    for sh in pgl_scene:
        if isinstance(sh.geometry, pgl.Text):
            continue
        shapes.append(sh)
    simulator.addFaceSensors(shapes)
    
    simulator.scene_pgl = pgl_scene + env
    simulator.scene = sp_scene

    # S2 series of lamps
    simulator.addPointLight(Vec3(730, 1670, 2100), 400)
    simulator.addPointLight(Vec3(730, 176, 2100), 400)
    simulator.addPointLight(Vec3(1670, 176, 2100), 400)
    simulator.addPointLight(Vec3(1670, 1660, 2100), 400)
    
    # run
    res = simulator.run()
    sc = simulator.visualizePhotons(mode="oawidgets")
    rays = simulator.visualizeRays(mode="oawidgets", color=0x33d17a, opacity=0.105)
    cmap = simulator.visualizeResults(mode='oawidgets')

    tab = Tab()
    tab.children = [sc, rays, cmap]
    tab.titles = ["Photons", "Rays", "Colormap"]
    display(tab)
    return simulator

opts = list(map(str, list(range(len(get_all_manips())))))
manips_opts = sorted(list(map(str, list(manips()))))
stage_opts = ["FBV","PCV","OF"]

In [ ]:
manip_name=Dropdown(options=manips_opts, value="High_Light_Crop_1")
stage=Dropdown(options=stage_opts, value="FBV")

def update_stages(*args):
    options = experiments(manip_name.value)
    options = [o.name for o in options] 
    stage.options = sorted(options)

manip_name.observe(update_stages, "value")
fill=Checkbox(value=False)

In [ ]:
w2 = interactive(select_experiment, {"manual": True}, manip_name=manip_name, stage=stage, fill=fill)
w2